In [2]:
import pandas as pd

# Importa el csv a un data frame
# Input(x) -> Comentarios (review)
# Output(y) -> Sentimientos
df_review = pd.read_csv('IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# Desbalanceamos el dataset

In [4]:
# Crea un set de 9000 comentarios positivos y 
# otro set 1000 comentarios negativos
df_positivo = df_review[df_review['sentiment']=='positive'][:9000]
df_negativo = df_review[df_review['sentiment']=='negative'][:1000]

df_review_des = pd.concat([df_positivo, df_negativo])
df_review_des.value_counts('sentiment')

sentiment
positive    9000
negative    1000
dtype: int64

# Balanceamos el dataset

In [14]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()
# La X tiene que ser un 2D array, por eso lleva doble []
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_des[['review']], df_review_des['sentiment'])
df_review_bal.value_counts('sentiment')

sentiment
negative    1000
positive    1000
dtype: int64

# Separar data para entrenar (train y test)

In [25]:
from sklearn.model_selection import train_test_split

# Usamos un 33% del df para testear. El parámetro random_state
# no es obligatorio, solamente sirve para que siempre que ejecutemos
# el código coja los mismos datos y no devuelva scores diferentes
train, test = train_test_split(df_review_bal, test_size=0.33, random_state=42)

In [44]:
# Definimos el input (x) y el output(y)
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

# Representación de Text (Bag of Words)

Transformamos los textos a data numérica (vectores) según la relevancia de cada palabra.
Hay dos técnicas famosas para hacer esta transformación:
- CountVectorizer (da peso según la cantidad de veces que aparece una palabra)
- Tfidf (da más peso a las palabras más representativas, aquellas que aparecen mucho en un documento pero no en otro)

# Transformar data de texto a data numérica

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

# Fit: encuentra los mejores parámetros para la data que introducimos
# Transform: aplica estos parámetros a la data que le pasamos
train_x_vector = tfidf.fit_transform(train_x)

# Aqui ya no hacemos fit porque tfidf ya tiene los valores óptimos,
# solamente los vamos a aplicar a test
test_x_vector = tfidf.transform(test_x)

# Selección del Modelo

ML algoritmos

1. Aprendizaje supervisado, input (review) y output (sentiment) predefinidos. Hay dos ramas:

- Regresion: output numérico
- Clasificación: output discreto (verdadero/falso, positivo/negativo, categoría, ...)

2. Aprendizaje no supervisado, trata de identificar patrones para predecir el output.

En este ejercicio vamos a crear 4 modelos: SVM, Decision Tree, Naive Bayes y Logistic Regression.

### Support Vector Machines (SVM)

In [49]:
from sklearn.svm import SVC

# svc = SVC()
svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y) # input numérico, output normal


SVC(kernel='linear')

In [50]:
# Hacemos predicciones con este modelo
print(svc.predict(tfidf.transform(['A good movie'])))
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['I did not like this movie at all I gave this movie away'])))


['positive']
['positive']
['negative']


### Decision Tree

In [51]:
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y) # input numérico, output normal

DecisionTreeClassifier()

In [52]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y) # input numérico como array, output normal

GaussianNB()

### Logistic Regression

In [53]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_x_vector, train_y) # input numérico, output normal

LogisticRegression()

# Evaluacion del Modelo

Para hacer la evaluación usamos los datos de test_*.

### Score (Accuracy)

In [66]:
print(svc.score(test_x_vector, test_y))
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y)) # input como array
print(lr.score(test_x_vector, test_y))

0.8287878787878787
0.6666666666666666
0.6060606060606061
0.8469696969696969


### F1 Score

A diferencia del Accuracy Score, este toma en cuenta cómo la data está distribuida. Si tenemos una data desbalanceada sería más conveniente tener en cuenta el F1 Score.

F1 Score = 2*(Recall * Precision) / (Recall + Precision)

In [67]:
from sklearn.metrics import f1_score

f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'], # orden de reporte
         average=None)

array([0.83455344, 0.82260597])

### Reporte de Clasificacion

In [69]:
from sklearn.metrics import classification_report

print(classification_report(test_y, svc.predict(test_x_vector),
                      labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.82      0.85      0.83       334
    negative       0.84      0.80      0.82       326

    accuracy                           0.83       660
   macro avg       0.83      0.83      0.83       660
weighted avg       0.83      0.83      0.83       660



### Confusion Matrix


In [70]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, svc.predict(test_x_vector),
                 labels=['positive', 'negative'])

array([[285,  49],
       [ 64, 262]])

# Optimización del Modelo

### GridSearchCV

In [71]:
from sklearn.model_selection import GridSearchCV

# C: cuánto error es soportable
# Kernel: qué tipo de función queremos usar (lineales, polinómicas, rbf)
# Se pueden agregar más parámetros...
parametros = {'C':[1, 4, 8, 16, 32], 'kernel':['linear', 'rbf']}

# GridSearch va a hacer una búsqueda exhaustiva de qué parmámetros, 
# dentro de todos los que hemos colocado, son los mejores para 
# nuestro modelo
svc = SVC()

# cv: cross validation (cuántas validaciones cruzadas se van a hacer)
# Hay más parámetros...
svc_grid = GridSearchCV(svc, parametros, cv=5)
svc_grid.fit(train_x_vector, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['linear', 'rbf']})

In [72]:
print(svc_grid.best_estimator_)
print(svc_grid.best_params_)

SVC(C=4)
{'C': 4, 'kernel': 'rbf'}


In [73]:
svc_grid.best_score_

0.8350746268656717